In [45]:
import numpy as np
import pandas as pd
import matplotlib
%matplotlib inline
import matplotlib.pyplot as matplot
from sklearn.decomposition import PCA
import random
from IPython.display import display, HTML
from itertools import chain
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import seaborn as sb
from sklearn.model_selection import ParameterGrid
from sklearn.svm import SVC, LinearSVC
import warnings
warnings.filterwarnings('ignore')

Q1(a)(i) 
Load MNIST dataset. After necessary data preparation, build a linear SVM classifier. (5 points)

In [2]:
import tensorflow as tf
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

2022-11-05 15:29:02.862040: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-05 15:29:03.006257: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-05 15:29:03.006282: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-05 15:29:03.048860: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-05 15:29:03.937639: W tensorflow/stream_executor/platform/de

In [3]:
print(train_images.shape)
print(test_images.shape)

(60000, 28, 28)
(10000, 28, 28)


In [4]:
train_images = train_images/255.0

In [5]:
test_images = test_images/255.0

In [6]:
train_images = np.reshape(train_images, (60000, 784))
test_images = np.reshape(test_images, (10000, 784))

In [7]:
print(train_images.shape)
print(test_images.shape)

(60000, 784)
(10000, 784)


In [8]:
svm = LinearSVC(dual=False)
svm.fit(train_images, train_labels)
pred = svm.predict(test_images)
accuracy_score(test_labels, pred) 

0.9184

ii. Show the predictions for first twenty values of your test dataset. Display the confusion matrix. (4
points)

In [9]:
for i in range(20):
    print(pred[i])

7
2
1
0
4
1
4
9
6
9
0
6
9
0
1
5
9
7
3
4


In [16]:
confmat = confusion_matrix(test_labels, pred)
print(confmat)

[[ 962    0    2    1    1    4    5    3    1    1]
 [   0 1112    3    2    0    1    5    1   11    0]
 [  11   11  914   18   10    4   13   12   36    3]
 [   4    0   19  918    2   22    5   12   20    8]
 [   1    4    5    4  913    0    9    3    5   38]
 [   9    2    0   39   12  769   17    7   29    8]
 [   7    4    7    2    5   21  909    1    2    0]
 [   2    8   23    5    7    1    1  948    4   29]
 [  11   13    8   21   14   31    8   13  843   12]
 [   7    8    2   15   31   12    0   26   12  896]]


Write your own function to calculate class-wise F1 score. 

In [17]:
type(confmat)

numpy.ndarray

In [36]:
from operator import le


prec_list = confmat.sum(axis=0)
recall_list = confmat.sum(axis=1)
precisions = []
recalls = []
for i in range(10):
    precisions.append(float(confmat[i][i])/prec_list[i])
    recalls.append(float(confmat[i][i])/recall_list[i])

print(precisions)
print(recalls)


[0.9487179487179487, 0.9569707401032702, 0.9298067141403866, 0.895609756097561, 0.9175879396984925, 0.8890173410404625, 0.9351851851851852, 0.9239766081871345, 0.8753894080996885, 0.9005025125628141]
[0.9816326530612245, 0.9797356828193833, 0.8856589147286822, 0.9089108910891089, 0.929735234215886, 0.8621076233183856, 0.9488517745302714, 0.9221789883268483, 0.8655030800821355, 0.88800792864222]


In [39]:
def f1(pr):
    (p,r) = pr
    return (2*(p*r)/(p+r))
f1s = list(map(f1, zip(precisions, recalls)))

print(f1s)

[0.9648946840521565, 0.9682194166303876, 0.907196029776675, 0.9022113022113023, 0.9236216489630754, 0.8753557199772338, 0.9419689119170985, 0.9230769230769231, 0.8704181724315951, 0.8942115768463074]


Check the F1 scores using sklearn inbuilt function and compare the value with the F1 scores returned
by your function written from scratch. Also, report the accuracy.

In [40]:
from sklearn.metrics import f1_score
f1s_scikit = f1_score(test_labels, pred, average=None)


In [41]:
print(f1s)
print(f1s_scikit)


[0.9648946840521565, 0.9682194166303876, 0.907196029776675, 0.9022113022113023, 0.9236216489630754, 0.8753557199772338, 0.9419689119170985, 0.9230769230769231, 0.8704181724315951, 0.8942115768463074]
[0.96489468 0.96821942 0.90719603 0.9022113  0.92362165 0.87535572
 0.94196891 0.92307692 0.87041817 0.89421158]


In [42]:
accuracy_score(test_labels, pred)

0.9184

(b) Non linear SVM:
Build non-linear models with the RBF kernel as well as polynomial kernel. Report the accuracy. (5 points)

In [43]:
svm = SVC(kernel='rbf')
svm.fit(train_images, train_labels)
pred = svm.predict(test_images)
print(accuracy_score(test_labels, pred) )

0.9792


In [44]:
svm = SVC(kernel='poly')
svm.fit(train_images, train_labels)
pred = svm.predict(test_images)
print(accuracy_score(test_labels, pred) )

0.9771


In [ ]:
C = [50, 10, 1.0, 0.1, 0.01]
G = [0.1, 1, 10]
grid = dict(C=C,G=G)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)